In [1]:
# Import data
import pandas as pd

df = pd.read_csv('../top_rated_wines.csv')

df.head()

,name,region,variety,rating,notes
0,3 Rings Reserve Shiraz 2004,"Barossa Valley, Barossa, South Australia, Aust...",Red Wine,96.0,Vintage Comments : Classic Barossa vintage con...
1,Abreu Vineyards Cappella 2007,"Napa Valley, California",Red Wine,96.0,Cappella is a proprietary blend of two clones ...
2,Abreu Vineyards Cappella 2010,"Napa Valley, California",Red Wine,98.0,Cappella is one of the oldest vineyard sites i...
3,Abreu Vineyards Howell Mountain 2008,"Howell Mountain, Napa Valley, California",Red Wine,96.0,When David purchased this Howell Mountain prop...
4,Abreu Vineyards Howell Mountain 2009,"Howell Mountain, Napa Valley, California",Red Wine,98.0,"As a set of wines, it is hard to surpass the f..."


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1365 entries, 0 to 1364
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   name     1365 non-null   object 
 1   region   1364 non-null   object 
 2   variety  1347 non-null   object 
 3   rating   1365 non-null   float64
 4   notes    1365 non-null   object 
dtypes: float64(1), object(4)
memory usage: 53.4+ KB


In [4]:
# remove any NaN values as it blows up serialization
df = df[df['variety'].notna()]
data = df.to_dict('records')
df.head()

/tmp/ipykernel_8260/4284288122.py:3: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  data = df.to_dict('record')


,name,region,variety,rating,notes
0,3 Rings Reserve Shiraz 2004,"Barossa Valley, Barossa, South Australia, Aust...",Red Wine,96.0,Vintage Comments : Classic Barossa vintage con...
1,Abreu Vineyards Cappella 2007,"Napa Valley, California",Red Wine,96.0,Cappella is a proprietary blend of two clones ...
2,Abreu Vineyards Cappella 2010,"Napa Valley, California",Red Wine,98.0,Cappella is one of the oldest vineyard sites i...
3,Abreu Vineyards Howell Mountain 2008,"Howell Mountain, Napa Valley, California",Red Wine,96.0,When David purchased this Howell Mountain prop...
4,Abreu Vineyards Howell Mountain 2009,"Howell Mountain, Napa Valley, California",Red Wine,98.0,"As a set of wines, it is hard to surpass the f..."


In [6]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

In [7]:
# Load the model to create embeddings
# .cache/torch/sentence_transformers/sentence-transformers_all-MiniLM-L6-v2/
encoder = SentenceTransformer('all-MiniLM-L6-v2')

.gitattributes:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [8]:
# create the vector database client
qdrant = QdrantClient(":memory:") # create in-memory Qdrant instance

In [10]:
# create vector config
vector_config = models.VectorParams(
    size = encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
    distance = models.Distance.COSINE
)

# create collection to store books
qdrant.recreate_collection(
    collection_name = 'top_wines',
    vectors_config = vector_config
)

True

In [18]:
# Try to convert notes to vector
print(data[0]['notes'])
print(encoder.encode(data[0]['notes']))

Vintage Comments : Classic Barossa vintage conditions. An average wet Spring followed by extreme heat in early February. Occasional rainfall events kept the vines in good balance up to harvest in late March 2004. Very good quality coupled with good average yields. More than 30 months in wood followed by six months tank maturation of the blend prior to bottling, July 2007. 
[-5.20038866e-02  2.14480162e-02  4.04233225e-02  4.26626727e-02
 -9.58252996e-02 -2.90006083e-02 -8.80133584e-02 -1.84591692e-02
 -8.99091437e-02  4.50307690e-02  6.64311287e-04 -3.42001766e-02
  9.54557396e-03  6.31257445e-02 -1.40816849e-02 -3.03313211e-02
  4.93896715e-02 -1.44043639e-01 -2.13793907e-02 -3.18222446e-03
  3.68515812e-02 -4.31906199e-03 -1.46433134e-02  6.09905971e-03
  8.33442658e-02 -5.51948659e-02 -6.00770526e-02  4.38864529e-02
  1.00697294e-01  1.32786371e-02  2.35872976e-02  8.34999681e-02
  3.42347212e-02 -4.39797379e-02  2.32254099e-02 -6.17142208e-02
  3.71921510e-02 -9.25690159e-02 -7.118

In [20]:
# vectorize!
qdrant.upsert(
    collection_name='top_wines',
    points = [ models.PointStruct(id = idx,
                                  vector = encoder.encode(doc['notes']).tolist(), 
                                  payload = doc
                                 ) 
              for idx, doc in enumerate(data) ]
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [27]:
# Searching

results = qdrant.search(
    collection_name='top_wines',
    query_vector=encoder.encode("99 points Cabernet Sauvignon from Napa Valley").tolist(),
    limit=3
)

for result in results:
    print(result.payload, 'score:',result.score)
    print('\n')

{'name': 'Kapcsandy Family Winery State Lane Cabernet Sauvignon Grand Vin 2017', 'region': 'Yountville, Napa Valley, California', 'variety': 'Red Wine', 'rating': 96.0, 'notes': '100% Cabernet Sauvignon'} score: 0.7492028788117284


{'name': 'Lewis Cellars Cabernet Sauvignon 2002', 'region': 'Napa Valley, California', 'variety': 'Red Wine', 'rating': 96.0, 'notes': 'Showcasing the unique personalities of small hillside vineyards from Pritchard Hill, Oakville and Rutherford, the 2002 Napa Valley Cabernet delivers compelling aromas of mocha, ripe berries, tobacco and sweet oak spice. The wine is 100% Cabernet Sauvignon, complex, rich and focused. With a deep core of black fruit and traces of briar and vanilla, it turns chocolaty and long on the palate with serious, integrated tannins.'} score: 0.7331373438014016


{'name': 'Anakota Helena Montana Vineyard Cabernet Sauvignon 2013', 'region': 'Knights Valley, Sonoma County, California', 'variety': 'Red Wine', 'rating': 96.0, 'notes': 'Blen

In [28]:
# Another searching

results = qdrant.search(
    collection_name='top_wines',
    query_vector=encoder.encode("A wine from Mendoza Argentina").tolist(),
    limit=3
)

for result in results:
    print(result.payload, 'score:',result.score)
    print('\n')

{'name': 'Catena Zapata Nicasia Vineyard Malbec 2004', 'region': 'Argentina', 'variety': 'Red Wine', 'rating': 96.0, 'notes': '"The single-vineyard 2004 Malbec Nicasia Vineyard is located in the Altamira district of Mendoza. It was aged for 18 months in new French oak. Opaque purple-colored, it exhibits a complex perfume of pain grille, scorched earth, mineral, licorice, blueberry, and black cherry. Thick on the palate, bordering on opulent, it has layers of fruit, silky tannins, and a long, fruit-filled finish. It will age effortlessly for another 6-8 years and provide pleasure through 2025. When all is said and done, Catena Zapata is the Argentina winery of reference – the standard of excellence for comparing all others. The brilliant, forward-thinking Nicolas Catena remains in charge, with his daughter, Laura, playing an increasingly large role. The Catena Zapata winery is an essential destination for fans of both architecture and wine in Mendoza. It is hard to believe, given the su